In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/test.csv
/kaggle/input/digit-recognizer/train.csv


In [4]:
X_test = pd.read_csv("../input/digit-recognizer/test.csv")
train = pd.read_csv("../input/digit-recognizer/train.csv")
Y_train = train["label"]
X_train = train.drop(labels=["label"],axis = 1)
del train

In [5]:
print(X_test.shape)
print(X_train.shape)
print(Y_train.shape)

(28000, 784)
(42000, 784)
(42000,)


In [6]:
X_train = X_train / 255.0
X_test = X_test / 255.0
X_train = X_train.values.reshape(-1,28,28,1)
X_test = X_test.values.reshape(-1,28,28,1)
print(X_train.shape)
print(X_test.shape)

(42000, 28, 28, 1)
(28000, 28, 28, 1)


In [8]:
from keras.utils.np_utils import to_categorical
Y_train = to_categorical(Y_train, num_classes=10)

In [9]:
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dropout, Dense, MaxPool2D
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

In [10]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.20, random_state=10)

In [11]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dense(10, activation = "softmax"))

In [12]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)

In [14]:
history = model.fit(datagen.flow(X_train, Y_train, batch_size=84), steps_per_epoch=(X_train.shape[0]//84), epochs=3, validation_data=(X_val, Y_val))

Epoch 1/3
400/400 [==============================] - 30s 75ms/step - loss: 0.3572 - accuracy: 0.8890 - val_loss: 0.0633 - val_accuracy: 0.9785
Epoch 2/3
400/400 [==============================] - 30s 75ms/step - loss: 0.1066 - accuracy: 0.9680 - val_loss: 0.0469 - val_accuracy: 0.9838
Epoch 3/3
400/400 [==============================] - 29s 72ms/step - loss: 0.0752 - accuracy: 0.9766 - val_loss: 0.0396 - val_accuracy: 0.9881


In [16]:
Y_pred = model.predict(X_val)
Y_pred_final = np.argmax(Y_pred, axis=1)
Y_true = np.argmax(Y_val, axis = 1)
confusion_matrix(Y_true, Y_pred_final)

array([[791,   0,   0,   0,   0,   0,   2,   0,   2,   1],
       [  0, 936,   4,   0,   2,   0,   1,   1,   2,   0],
       [  0,   0, 849,   0,   3,   0,   0,   1,   3,   0],
       [  1,   0,   1, 845,   0,   6,   0,   3,   0,   0],
       [  0,   0,   1,   0, 821,   0,   1,   0,   1,   2],
       [  0,   0,   1,   1,   0, 769,   2,   0,   0,   0],
       [  0,   0,   0,   0,   1,   2, 784,   0,   0,   0],
       [  0,   0,   9,   0,   3,   1,   0, 853,   0,   3],
       [  0,   3,   2,   0,   3,   7,   7,   0, 810,   4],
       [  0,   0,   0,   0,   7,   3,   0,   0,   3, 842]])

In [18]:
res = model.predict(X_test)
res

array([[3.2365829e-08, 1.5916972e-09, 9.9999976e-01, ..., 1.4509330e-07,
        9.7326751e-09, 1.5742511e-10],
       [9.9560601e-01, 1.4617977e-09, 1.3582978e-03, ..., 4.5964686e-07,
        1.5730307e-04, 2.6688224e-03],
       [7.9411842e-07, 3.2428019e-08, 5.1472962e-05, ..., 1.4415031e-05,
        5.2884058e-03, 9.9429673e-01],
       ...,
       [1.3124021e-13, 2.1032393e-11, 6.0347358e-09, ..., 1.3664977e-09,
        6.7111863e-08, 3.9418953e-08],
       [6.3157785e-07, 1.8806512e-08, 8.5641920e-09, ..., 1.3728549e-04,
        3.1986821e-07, 9.9820852e-01],
       [1.5692212e-08, 3.9878068e-10, 9.9999905e-01, ..., 8.9249488e-09,
        9.7111501e-07, 6.4729884e-09]], dtype=float32)

In [19]:
res

array([[3.2365829e-08, 1.5916972e-09, 9.9999976e-01, ..., 1.4509330e-07,
        9.7326751e-09, 1.5742511e-10],
       [9.9560601e-01, 1.4617977e-09, 1.3582978e-03, ..., 4.5964686e-07,
        1.5730307e-04, 2.6688224e-03],
       [7.9411842e-07, 3.2428019e-08, 5.1472962e-05, ..., 1.4415031e-05,
        5.2884058e-03, 9.9429673e-01],
       ...,
       [1.3124021e-13, 2.1032393e-11, 6.0347358e-09, ..., 1.3664977e-09,
        6.7111863e-08, 3.9418953e-08],
       [6.3157785e-07, 1.8806512e-08, 8.5641920e-09, ..., 1.3728549e-04,
        3.1986821e-07, 9.9820852e-01],
       [1.5692212e-08, 3.9878068e-10, 9.9999905e-01, ..., 8.9249488e-09,
        9.7111501e-07, 6.4729884e-09]], dtype=float32)

In [20]:
results = np.argmax(res,axis = 1)

In [21]:
results


array([2, 0, 9, ..., 3, 9, 2])

In [23]:
results = pd.Series(results,name="Label")

In [24]:
test_output = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)
test_output.to_csv("test_output.csv", index=False)